In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
import pathlib
import pickle

sys.path.append('..')

In [3]:
import pandas as pd

In [4]:
from pals.pimp_tools import get_pimp_API_token_from_env, PIMP_HOST, download_from_pimp
from pals.feature_extraction import DataSource
from pals.pathway_analysis import PALS
from pals.common import *

2019-12-18 13:14:29.216 | WARNING  | pals.reactome:get_neo4j_driver:14 - Using a default neo4j server: bolt://localhost:7687
2019-12-18 13:14:29.218 | WARNING  | pals.reactome:get_neo4j_driver:19 - Using a default neo4j username or password: neo4j
2019-12-18 13:14:30.321 | INFO     | pals.reactome:get_neo4j_driver:24 - Created graph database driver for bolt://localhost:7687 (neo4j)


# Beer Analysis

### Load data

Generate token by logging in to PiMP

In [5]:
# username = 'joewandy' # PiMP username
# password = 'enter' # PiMP password
# host = 'localhost:8000' # server address and port
# token = get_authentication_token(host, username, password)

Assume token is stored in environmental variable *PIMP_API_TOKEN*

In [6]:
token = get_pimp_API_token_from_env()

In [7]:
analysis_id = 1321 # example beer analysis

In [8]:
int_df, annotation_df, experimental_design = download_from_pimp(token, PIMP_HOST, analysis_id, 'kegg')

2019-12-18 13:14:34.317 | DEBUG    | pals.pimp_tools:download_from_pimp:119 - Trying to load data from temp file: C:\Users\joewa\AppData\Local\Temp\pimp_analysis_1321.p
2019-12-18 13:14:34.318 | WARNING  | pals.common:load_obj:91 - Old, invalid or missing pickle in C:\Users\joewa\AppData\Local\Temp\pimp_analysis_1321.p. Please regenerate this file.
2019-12-18 13:14:34.318 | DEBUG    | pals.pimp_tools:download_from_pimp:123 - Retrieving data for analysis 1321 from PiMP
2019-12-18 13:14:35.865 | DEBUG    | pals.pimp_tools:get_data:33 - http://polyomics.mvls.gla.ac.uk/export/get_ms1_intensities?analysis_id=1321
2019-12-18 13:14:56.079 | DEBUG    | pals.pimp_tools:get_data:33 - http://polyomics.mvls.gla.ac.uk/export/get_ms1_peaks?analysis_id=1321
2019-12-18 13:15:08.793 | DEBUG    | pals.pimp_tools:get_data:33 - http://polyomics.mvls.gla.ac.uk/export/get_experimental_design?analysis_id=1321
2019-12-18 13:15:08.795 | DEBUG    | pals.pimp_tools:download_from_pimp:132 - Caching analysis data 

In [9]:
int_df

,Beer_1_full1.mzXML,Beer_1_full2.mzXML,Beer_1_full3.mzXML,Beer_2_full1.mzXML,Beer_2_full2.mzXML,Beer_2_full3.mzXML,Beer_3_full1.mzXML,Beer_3_full2.mzXML,Beer_3_full3.mzXML,Beer_4_full1.mzXML,Beer_4_full2.mzXML,Beer_4_full3.mzXML
row_id,,,,,,,,,,,,
3033929,2.235291e+09,2.000478e+09,2.170697e+09,2.242760e+09,2.279882e+09,1.959480e+09,2.079356e+09,2.110473e+09,2.243653e+09,1.817065e+09,1.746443e+09,1.779827e+09
3033930,4.433491e+07,4.287387e+07,4.894853e+07,4.760448e+07,4.217280e+07,3.908452e+07,3.825778e+07,3.770192e+07,4.087189e+07,3.330477e+07,3.153630e+07,3.102410e+07
3033931,1.723985e+09,1.764235e+09,1.585143e+09,1.543961e+09,1.579320e+09,1.555666e+09,1.698130e+09,1.481824e+09,1.508645e+09,1.642510e+09,1.723919e+09,1.697806e+09
3033932,6.254237e+08,6.503417e+08,5.914975e+08,4.635929e+08,4.298382e+08,4.038747e+08,4.292837e+08,3.708761e+08,4.778932e+08,3.903165e+08,4.080995e+08,4.309892e+08
3033933,1.075022e+09,9.293474e+08,1.092635e+09,1.130720e+09,1.118146e+09,1.192834e+09,1.231442e+09,1.262046e+09,1.460653e+09,1.009838e+09,9.085111e+08,9.967176e+08
...,...,...,...,...,...,...,...,...,...,...,...,...
3041299,1.431211e+04,6.565678e+03,1.478325e+04,1.620252e+04,1.748920e+04,1.284756e+04,3.306687e+04,2.476216e+04,2.869417e+04,2.231166e+04,2.164017e+04,2.727751e+04
3041300,2.273721e+04,2.905976e+04,2.756565e+04,3.080164e+04,2.427240e+04,2.517871e+04,2.718543e+04,2.905361e+04,3.170420e+04,2.597168e+04,3.066904e+04,2.884984e+04
3041301,1.760107e+04,2.674373e+04,2.165889e+04,2.121242e+04,1.737357e+04,2.039137e+04,2.296079e+04,2.001743e+04,2.664124e+04,1.791359e+04,1.642459e+04,2.349759e+04


In [10]:
annotation_df

,entity_id
row_id,
3033929,C00148
3036581,C00148
3036855,C00148
3038249,C00148
3033929,C00163
...,...
3040926,C20522
3040929,C20582
3041077,C20499


In [11]:
experimental_design

{'comparisons': [{'case': 'beer1', 'control': 'beer2', 'name': 'beer1/beer2'},
  {'case': 'beer3', 'control': 'beer4', 'name': 'beer3/beer4'}],
 'groups': {'beer4': ['Beer_4_full3.mzXML',
   'Beer_4_full2.mzXML',
   'Beer_4_full1.mzXML'],
  'beer3': ['Beer_3_full3.mzXML', 'Beer_3_full2.mzXML', 'Beer_3_full1.mzXML'],
  'beer2': ['Beer_2_full3.mzXML', 'Beer_2_full1.mzXML', 'Beer_2_full2.mzXML'],
  'beer1': ['Beer_1_full2.mzXML', 'Beer_1_full1.mzXML', 'Beer_1_full3.mzXML']}}

### PALS analysis using KEGG database exported from PiMP

In [12]:
ds = DataSource(int_df, annotation_df, experimental_design, DATABASE_PIMP_KEGG)

2019-12-18 13:29:54.631 | DEBUG    | pals.feature_extraction:get_data:96 - Loading C:\Users\joewa\Work\git\PALS\pals\data\PiMP_KEGG.json.zip
2019-12-18 13:29:54.657 | DEBUG    | pals.feature_extraction:__init__:46 - Mapping pathway to unique ids
2019-12-18 13:29:54.663 | DEBUG    | pals.feature_extraction:__init__:60 - Creating dataset to pathway mapping
2019-12-18 13:29:55.564 | DEBUG    | pals.feature_extraction:__init__:88 - Computing unique id counts


In [13]:
pals = PALS(ds, min_replace=5000, plage_weight=5, hg_weight=1)
pathway_df = pals.get_pathway_df()

2019-12-18 13:29:55.640 | DEBUG    | pals.pathway_analysis:_change_zero_peak_ints:385 - Setting the zero intensity values in the dataframe
2019-12-18 13:29:55.670 | DEBUG    | pals.pathway_analysis:_standardize_intensity_df:366 - Scaling the data across the sample: zero mean and unit variance
2019-12-18 13:29:55.677 | DEBUG    | pals.pathway_analysis:get_plage_activity_df:205 - Mean values of the rows in the DF is [ 0.  0. -0. ... -0. -0. -0.]
2019-12-18 13:29:55.678 | DEBUG    | pals.pathway_analysis:get_plage_activity_df:206 - Variance in the rows of the DF is [1. 1. 1. ... 1. 1. 1.]
2019-12-18 13:29:56.386 | DEBUG    | pals.pathway_analysis:set_up_resample_plage_p_df:217 - Calculating plage p-values with resampling
2019-12-18 13:29:56.387 | DEBUG    | pals.pathway_analysis:set_up_resample_plage_p_df:221 - Comparison beer1/beer2
2019-12-18 13:29:56.387 | DEBUG    | pals.pathway_analysis:set_up_resample_plage_p_df:229 - Resampling 0/1000
2019-12-18 13:29:56.486 | DEBUG    | pals.pathw

In [14]:
pathway_df.sort_values('PiMP_KEGG beer1/beer2 comb_p', ascending=True, inplace=True)
pathway_df

,pw_name,beer1/beer2 p-value,beer3/beer4 p-value,unq_pw_F,tot_ds_F,F_coverage,sf,exp_F,Ex_Cov,PiMP_KEGG beer1/beer2 comb_p,PiMP_KEGG beer3/beer4 comb_p
map00380,Tryptophan metabolism,0.026410,0.050835,64,34,53.12,2.024112e-08,14.26,22.28,0.001464,0.003665
map00330,Arginine and proline metabolism,0.076544,0.031235,79,50,63.29,7.494293e-16,17.61,22.29,0.001512,0.000348
map00460,Cyanoamino acid metabolism,0.033922,0.042044,40,25,62.50,1.448042e-08,8.92,22.30,0.001998,0.002702
map00300,Lysine biosynthesis,0.037750,0.034624,27,20,74.07,3.670262e-09,6.02,22.30,0.002007,0.001775
map00400,"Phenylalanine, tyrosine and tryptophan biosynt...",0.051530,0.031751,30,22,73.33,9.098577e-10,6.69,22.30,0.002736,0.001355
...,...,...,...,...,...,...,...,...,...,...,...
map07227,Histamine H2/H3 receptor agonists/antagonists,1.000000,0.212082,10,1,10.00,7.411824e-01,2.23,22.30,1.000000,0.255651
map00523,Polyketide sugar unit biosynthesis,1.000000,0.053513,19,2,10.53,8.566036e-01,4.23,22.26,1.000000,0.085109
map05032,Morphine addiction,1.000000,0.071093,8,3,37.50,1.185365e-01,1.78,22.25,1.000000,0.047353
map01040,Biosynthesis of unsaturated fatty acids,1.000000,0.069916,42,4,9.52,9.770476e-01,9.36,22.29,1.000000,0.145428


### PALS analysis using KEGG database exported from Reactome

In [15]:
ds = DataSource(int_df, annotation_df, experimental_design, DATABASE_REACTOME_KEGG, 
                reactome_species=REACTOME_SPECIES_HOMO_SAPIENS, reactome_metabolic_pathway_only=True)

2019-12-18 13:29:59.211 | DEBUG    | pals.feature_extraction:get_data:120 - Loading ..\pals\data\reactome\metabolic_pathways\COMPOUND\Homo sapiens.json.zip
2019-12-18 13:29:59.234 | DEBUG    | pals.feature_extraction:__init__:46 - Mapping pathway to unique ids
2019-12-18 13:29:59.236 | DEBUG    | pals.feature_extraction:__init__:60 - Creating dataset to pathway mapping
2019-12-18 13:30:00.161 | DEBUG    | pals.feature_extraction:__init__:88 - Computing unique id counts


In [16]:
pals = PALS(ds, min_replace=5000, plage_weight=5, hg_weight=1)
pathway_df = pals.get_pathway_df()

2019-12-18 13:30:00.247 | DEBUG    | pals.pathway_analysis:_change_zero_peak_ints:385 - Setting the zero intensity values in the dataframe
2019-12-18 13:30:00.276 | DEBUG    | pals.pathway_analysis:_standardize_intensity_df:366 - Scaling the data across the sample: zero mean and unit variance
2019-12-18 13:30:00.285 | DEBUG    | pals.pathway_analysis:get_plage_activity_df:205 - Mean values of the rows in the DF is [ 0.  0. -0. ... -0. -0. -0.]
2019-12-18 13:30:00.286 | DEBUG    | pals.pathway_analysis:get_plage_activity_df:206 - Variance in the rows of the DF is [1. 1. 1. ... 1. 1. 1.]
2019-12-18 13:30:00.725 | DEBUG    | pals.pathway_analysis:set_up_resample_plage_p_df:217 - Calculating plage p-values with resampling
2019-12-18 13:30:00.726 | DEBUG    | pals.pathway_analysis:set_up_resample_plage_p_df:221 - Comparison beer1/beer2
2019-12-18 13:30:00.727 | DEBUG    | pals.pathway_analysis:set_up_resample_plage_p_df:229 - Resampling 0/1000
2019-12-18 13:30:00.852 | DEBUG    | pals.pathw

In [17]:
pathway_df.sort_values('COMPOUND beer1/beer2 comb_p', ascending=True, inplace=True)
pathway_df

,pw_name,beer1/beer2 p-value,beer3/beer4 p-value,unq_pw_F,tot_ds_F,F_coverage,sf,exp_F,Ex_Cov,COMPOUND beer1/beer2 comb_p,COMPOUND beer3/beer4 comb_p
R-HSA-71240,Tryptophan catabolism,0.016712,0.057485,27,14,51.85,0.020106,9.12,33.78,0.006423,0.025709
R-HSA-2024096,HS-GAG degradation,0.011545,0.078462,5,1,20.00,0.659417,1.69,33.80,0.015887,0.095534
R-HSA-163685,Integration of energy metabolism,0.032313,1.000000,1,1,100.00,0.113509,0.34,34.00,0.020237,1.000000
R-HSA-351143,Agmatine biosynthesis,0.023579,0.115855,5,2,40.00,0.327631,1.69,33.80,0.020981,0.103781
R-HSA-71182,Phenylalanine and tyrosine catabolism,0.051213,0.131596,24,13,54.17,0.015294,8.11,33.79,0.021405,0.064101
...,...,...,...,...,...,...,...,...,...,...,...
R-HSA-1989781,PPARA activates gene expression,1.000000,0.341515,2,1,50.00,0.264649,0.68,34.00,1.000000,0.300218
R-HSA-2046106,alpha-linolenic acid (ALA) metabolism,1.000000,0.094125,14,1,7.14,0.983784,4.73,33.79,1.000000,0.191954
R-HSA-1855183,"Synthesis of IP2, IP, and Ins in the cytosol",1.000000,0.064520,11,3,27.27,0.622386,3.72,33.82,1.000000,0.076749
R-HSA-70221,Glycogen breakdown (glycogenolysis),1.000000,0.064520,10,3,30.00,0.541649,3.38,33.80,1.000000,0.071064


### PALS analysis of compounds by connecting to Reactome

In [18]:
ds = DataSource(int_df, annotation_df, experimental_design, DATABASE_REACTOME_KEGG, 
                reactome_species=REACTOME_SPECIES_HOMO_SAPIENS, reactome_metabolic_pathway_only=True, reactome_query=True)

2019-12-18 13:30:03.748 | DEBUG    | pals.feature_extraction:get_data:102 - Retrieving data for Homo sapiens from Reactome COMPOUND metabolic_pathway_only=True
2019-12-18 13:30:08.208 | DEBUG    | pals.feature_extraction:__init__:46 - Mapping pathway to unique ids
2019-12-18 13:30:08.210 | DEBUG    | pals.feature_extraction:__init__:60 - Creating dataset to pathway mapping
2019-12-18 13:30:09.122 | DEBUG    | pals.feature_extraction:__init__:88 - Computing unique id counts


In [19]:
pals = PALS(ds, min_replace=5000, plage_weight=5, hg_weight=1)
pathway_df = pals.get_pathway_df()

2019-12-18 13:30:09.200 | DEBUG    | pals.pathway_analysis:_change_zero_peak_ints:385 - Setting the zero intensity values in the dataframe
2019-12-18 13:30:09.229 | DEBUG    | pals.pathway_analysis:_standardize_intensity_df:366 - Scaling the data across the sample: zero mean and unit variance
2019-12-18 13:30:09.238 | DEBUG    | pals.pathway_analysis:get_plage_activity_df:205 - Mean values of the rows in the DF is [ 0.  0. -0. ... -0. -0. -0.]
2019-12-18 13:30:09.239 | DEBUG    | pals.pathway_analysis:get_plage_activity_df:206 - Variance in the rows of the DF is [1. 1. 1. ... 1. 1. 1.]
2019-12-18 13:30:09.645 | DEBUG    | pals.pathway_analysis:set_up_resample_plage_p_df:217 - Calculating plage p-values with resampling
2019-12-18 13:30:09.646 | DEBUG    | pals.pathway_analysis:set_up_resample_plage_p_df:221 - Comparison beer1/beer2
2019-12-18 13:30:09.647 | DEBUG    | pals.pathway_analysis:set_up_resample_plage_p_df:229 - Resampling 0/1000
2019-12-18 13:30:09.741 | DEBUG    | pals.pathw

In [20]:
pathway_df.sort_values('COMPOUND beer1/beer2 comb_p', ascending=True, inplace=True)
pathway_df

,pw_name,beer1/beer2 p-value,beer3/beer4 p-value,unq_pw_F,tot_ds_F,F_coverage,sf,exp_F,Ex_Cov,COMPOUND beer1/beer2 comb_p,COMPOUND beer3/beer4 comb_p
R-HSA-71240,Tryptophan catabolism,0.025106,0.053509,27,14,51.85,0.020106,9.12,33.78,0.010104,0.023696
R-HSA-2024096,HS-GAG degradation,0.018223,0.066147,5,1,20.00,0.659417,1.69,33.80,0.024378,0.081462
R-HSA-163685,Integration of energy metabolism,0.044891,1.000000,1,1,100.00,0.113509,0.34,34.00,0.028682,1.000000
R-HSA-71182,Phenylalanine and tyrosine catabolism,0.069151,0.115252,24,13,54.17,0.015294,8.11,33.79,0.030229,0.054815
R-HSA-351143,Agmatine biosynthesis,0.034558,0.100506,5,2,40.00,0.327631,1.69,33.80,0.030748,0.089895
...,...,...,...,...,...,...,...,...,...,...,...
R-HSA-351200,Interconversion of polyamines,1.000000,0.926579,9,1,11.11,0.904169,3.04,33.78,1.000000,0.953391
R-HSA-2408517,SeMet incorporation into proteins,1.000000,0.080412,3,1,33.33,0.415382,1.01,33.67,1.000000,0.078244
R-HSA-164378,PKA activation in glucagon signalling,1.000000,0.080412,5,1,20.00,0.659417,1.69,33.80,1.000000,0.097748
R-HSA-77289,Mitochondrial Fatty Acid Beta-Oxidation,1.000000,0.080412,8,1,12.50,0.866228,2.70,33.75,1.000000,0.123507


### ORA Analysis

In [21]:
ds = DataSource(int_df, annotation_df, experimental_design, DATABASE_PIMP_KEGG)

2019-12-18 13:30:12.351 | DEBUG    | pals.feature_extraction:get_data:96 - Loading C:\Users\joewa\Work\git\PALS\pals\data\PiMP_KEGG.json.zip
2019-12-18 13:30:12.406 | DEBUG    | pals.feature_extraction:__init__:46 - Mapping pathway to unique ids
2019-12-18 13:30:12.412 | DEBUG    | pals.feature_extraction:__init__:60 - Creating dataset to pathway mapping
2019-12-18 13:30:13.446 | DEBUG    | pals.feature_extraction:__init__:88 - Computing unique id counts


In [22]:
pals = PALS(ds, min_replace=5000)
pathway_df = pals.get_ora_df()

2019-12-18 13:30:13.526 | DEBUG    | pals.pathway_analysis:get_ora_df:50 - Calculating ORA
2019-12-18 13:30:13.528 | DEBUG    | pals.pathway_analysis:_change_zero_peak_ints:385 - Setting the zero intensity values in the dataframe
2019-12-18 13:30:19.806 | DEBUG    | pals.pathway_analysis:get_ora_df:109 - Correcting for multiple t-tests
2019-12-18 13:30:19.812 | DEBUG    | pals.pathway_analysis:_calculate_coverage_df:471 - Calculating dataset formula coverage


In [23]:
pathway_df.sort_values('PiMP_KEGG beer1/beer2 comb_p', ascending=True, inplace=True)
pathway_df

,pw_name,beer1/beer2 p-value,beer3/beer4 p-value,PiMP_KEGG beer1/beer2 comb_p,PiMP_KEGG beer3/beer4 comb_p,unq_pw_F,tot_ds_F,F_coverage
mapids,,,,,,,,
map00350,Tyrosine metabolism,3.621191e-16,1.681454e-16,8.183892e-14,3.800087e-14,53,39,73.58
map00330,Arginine and proline metabolism,1.775486e-12,1.686603e-12,2.006299e-10,1.905861e-10,79,50,63.29
map00290,"Valine, leucine and isoleucine biosynthesis",9.591121e-12,2.028069e-11,7.225312e-10,1.527812e-09,17,16,94.12
map00400,"Phenylalanine, tyrosine and tryptophan biosynt...",2.824918e-10,7.042130e-10,1.596078e-08,3.978804e-08,30,22,73.33
map00660,C5-Branched dibasic acid metabolism,5.693949e-09,1.159169e-08,1.838332e-07,2.619722e-07,21,16,76.19
...,...,...,...,...,...,...,...,...
map00254,Aflatoxin biosynthesis,9.905000e-01,9.925788e-01,1.000000e+00,1.000000e+00,24,3,12.50
map00195,Photosynthesis,1.000000e+00,8.696482e-01,1.000000e+00,9.839475e-01,10,1,10.00
map00981,Insect hormone biosynthesis,9.829573e-01,1.000000e+00,1.000000e+00,1.000000e+00,21,2,9.52
